In [1]:
import networkx as nx
import torch
import torch.nn as nn
import numpy as np

from sklearn.metrics import average_precision_score

In [2]:
def makeG(n,m,p):
    M = np.zeros((n, m))
    G = nx.bipartite.random_graph(n,m,p)
    observed = list(G.edges())
    nodes = list(G.nodes())
    for i in range(n):
        for j in range(n, m + n):
            if (i,j) in observed:
                M[i][j - n] = 1
            else:
                M[i][j - n] = 0

                    
    return M

In [3]:
m,n,p = 10,10,.4

In [4]:
M = makeG(m,n,p)

In [5]:
M

array([[0., 1., 0., 1., 1., 0., 0., 0., 1., 1.],
       [1., 0., 0., 0., 0., 1., 0., 0., 0., 1.],
       [0., 0., 1., 1., 0., 0., 0., 0., 0., 1.],
       [0., 0., 1., 1., 0., 0., 0., 0., 0., 1.],
       [0., 1., 1., 0., 0., 0., 1., 0., 0., 1.],
       [0., 1., 0., 0., 0., 0., 0., 1., 1., 0.],
       [0., 0., 0., 1., 0., 0., 1., 1., 1., 0.],
       [0., 1., 0., 1., 0., 0., 1., 1., 0., 0.],
       [1., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 1., 1., 0., 1., 0.]])

# Model

In [6]:
class NMF(nn.Module):
    def __init__(self, M, N, K):
        super(NMF, self).__init__()
        self.A = nn.Parameter(torch.rand(M, K, requires_grad=True))
        self.S = nn.Parameter(torch.rand(K, N, requires_grad=True))
        self.logistic = nn.Sigmoid()

    def forward(self):
        return self.logistic(torch.matmul(self.A, self.S))

# Train

In [7]:
def train(model, crit, M, lr):
    model.train()
    model.zero_grad()
    pred = model()
    loss = crit(pred, M)
    loss.backward()
#     print(optim.parameters())
    for param in model.parameters():
        param.data -= lr * param.grad
    return loss

In [8]:
model = NMF(m,n,20)
# optim = torch.optim.SGD(model.parameters(), lr=.08, momentum=.9)
optim = torch.optim.Adam(model.parameters(), lr=.01)
crit = nn.BCELoss()

In [9]:
for epoch in range(15):
    l = train(model, crit, torch.from_numpy(M).float(), .1)
    if epoch % 10 == 0:
        print(l)

tensor(3.2977, grad_fn=<BinaryCrossEntropyBackward>)
tensor(2.8707, grad_fn=<BinaryCrossEntropyBackward>)


In [10]:
model.eval()
torch.round(model())

tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]], grad_fn=<RoundBackward>)

In [11]:
M

array([[0., 1., 0., 1., 1., 0., 0., 0., 1., 1.],
       [1., 0., 0., 0., 0., 1., 0., 0., 0., 1.],
       [0., 0., 1., 1., 0., 0., 0., 0., 0., 1.],
       [0., 0., 1., 1., 0., 0., 0., 0., 0., 1.],
       [0., 1., 1., 0., 0., 0., 1., 0., 0., 1.],
       [0., 1., 0., 0., 0., 0., 0., 1., 1., 0.],
       [0., 0., 0., 1., 0., 0., 1., 1., 1., 0.],
       [0., 1., 0., 1., 0., 0., 1., 1., 0., 0.],
       [1., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 1., 1., 0., 1., 0.]])

In [12]:
average_precision_score(M, model().detach().numpy())

0.5667156084656084